比赛网址：https://www.kaggle.com/c/porto-seguro-safe-driver-prediction

In [29]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

## 加载数据

In [30]:
df_train=pd.read_csv('D:\\Dataset\\Kaggle\\Porto Seguros Safe Driver Prediction\\train.csv')
df_train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [31]:
y=df_train['target'].values
df_train.drop(['id', 'target'], inplace=True, axis=1)
X=df_train.values

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2,stratify=y)

## 创建lgb数据

In [33]:
categorical_features = [c for c, col in enumerate(df_train.columns) if 'cat' in col]
print(categorical_features)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
test_data = lgb.Dataset(X_test, label=y_test)

[1, 3, 4, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


## 训练模型


In [34]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

model = lgb.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

E:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1158: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
E:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1161: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is [1, 3, 4, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
E:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's auc: 0.587454
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.600187
[3]	valid_0's auc: 0.604699
[4]	valid_0's auc: 0.611374
[5]	valid_0's auc: 0.613078
[6]	valid_0's auc: 0.614256
[7]	valid_0's auc: 0.615208
[8]	valid_0's auc: 0.616592
[9]	valid_0's auc: 0.618192
[10]	valid_0's auc: 0.618957
[11]	valid_0's auc: 0.619668
[12]	valid_0's auc: 0.620406
[13]	valid_0's auc: 0.620509
[14]	valid_0's auc: 0.620655
[15]	valid_0's auc: 0.620844
[16]	valid_0's auc: 0.621258
[17]	valid_0's auc: 0.621498
[18]	valid_0's auc: 0.622105
[19]	valid_0's auc: 0.621989
[20]	valid_0's auc: 0.621608
[21]	valid_0's auc: 0.621841
[22]	valid_0's auc: 0.622693
[23]	valid_0's auc: 0.622832
[24]	valid_0's auc: 0.622936
[25]	valid_0's auc: 0.623135
[26]	valid_0's auc: 0.623084
[27]	valid_0's auc: 0.622904
[28]	valid_0's auc: 0.623089
[29]	valid_0's auc: 0.623531
[30]	valid_0's auc: 0.623823
[31]	valid_0's auc: 0.624303
[32]	valid_0's auc: 0.624457
[33]	valid_0's auc

## 预测数据

In [35]:
submission = pd.read_csv('D:\\Dataset\\Kaggle\\Porto Seguros Safe Driver Prediction\\test.csv')
ids = submission['id'].values
submission.drop('id', inplace=True, axis=1)


x = submission.values
y = model.predict(x)

output = pd.DataFrame({'id': ids, 'target': y})
output.to_csv("submission.csv", index=False)